# NBA Analytics 360

## Importing necessary libraries:

In [7]:
import pandas as pd
import requests
import json

import time

import psycopg2
import psycopg2.extras as extras

## Data Collection from BALLDONTLIE API : 

### main function for data fetching:

In [8]:
def fetch_data_with_counter(data_type, url="https://www.balldontlie.io/api/v1", per_page=100, counter=None, last_counter=None):
    data = []
    requests_made = 0
    start_time = time.time()
    #loading value of final_counter to continue extraction from where we left:
    with open("final_counter.txt",'r') as f : 
        file_content = json.load(f)
    if counter is None:
            final_counter = (file_content[f"{data_type}_final_counter"])
            counter = final_counter + 1

    if last_counter is None:
        #while loop fetching all data available : 
        while type(counter) is int : 
            # GET request to the API with the specified URL and parameters stored in "call" variable :
            call = requests.get(f"{url}/{data_type}?per_page={per_page}&page={counter}")
            requests_made += 1
            # Extend the "data" list with retrieved data from API
            data.extend(call.json()["data"])

            # update "final_counter" if the "next_page" value is not an integer and the "data" list is not empty
            if call.json()["meta"]["next_page"] is not int and len(data)>0:
                final_counter = counter

            counter = call.json()["meta"]["next_page"]
            if requests_made == 60:
                print(len(games_data))
                elapsed_time = time.time() - start_time
                if elapsed_time < 60:
                    time.sleep(60 - elapsed_time)
                start_time = time.time()
                requests_made = 0
    else : 
        #while loop fetching all data available : 
        while counter<=last_counter : 
            # GET request to the API with the specified URL and parameters stored in "call" variable :
            call = requests.get(f"{url}/{data_type}?per_page={per_page}&page={counter}")
            # Extend the "data" list with retrieved data from the API
            data.extend(call.json()["data"])

            # update "final_counter" if the "next_page" value is not an integer and the "data" list is not empty
            if call.json()["meta"]["next_page"] is not int and len(data)>0:
                final_counter = counter

            counter = call.json()["meta"]["next_page"]
            
    file_content[f"{data_type}_final_counter"] = final_counter

    # update final_counter_file:
    with open("final_counter.txt", "w") as f: 
        json.dump(file_content, f)
    
    # turn json object into data_frame :
    df = pd.json_normalize(data)
    meta = call.json()["meta"]
    return df, meta

### PLAYERS DATA :

In [10]:
# players_df = fetch_data_with_counter(data_type="players")[0]
# players_df = players_df[["id", "first_name" , "last_name", "position", "height_feet", "height_inches", "weight_pounds", "team.id", "team.abbreviation", "team.city", "team.conference", "team.division", "team.name", "team.full_name"]]
players_df = pd.read_csv("api_data/player_data.csv", sep=";")

In [15]:
players_df.dtypes

id                     int64
first_name            object
height_feet          float64
height_inches        float64
last_name             object
position              object
weight_pounds        float64
team.id                int64
team.abbreviation     object
team.city             object
team.conference       object
team.division         object
team.full_name        object
team.name             object
dtype: object

### TEAMS DATA :

In [11]:
#teams_df = fetch_data_with_counter(data_type="teams")[0]
teams_df = pd.read_csv("api_data/teams_data.csv", sep=";")

In [16]:
teams_df.dtypes

id               int64
abbreviation    object
city            object
conference      object
division        object
full_name       object
name            object
dtype: object

### GAMES DATA :

In [12]:
#games_df = fetch_data_with_counter(data_type="games")[0]
games_df = pd.read_csv("api_data/games_data.csv", sep=";")

In [17]:
games_df.dtypes

id                            int64
date                         object
home_team_score               int64
period                        int64
postseason                     bool
season                        int64
status                       object
time                         object
visitor_team_score            int64
home_team.id                  int64
home_team.abbreviation       object
home_team.city               object
home_team.conference         object
home_team.division           object
home_team.full_name          object
home_team.name               object
visitor_team.id               int64
visitor_team.abbreviation    object
visitor_team.city            object
visitor_team.conference      object
visitor_team.division        object
visitor_team.full_name       object
visitor_team.name            object
dtype: object

In [28]:
list(e.replace(".", "_") for e in games_df.columns)

['id',
 'date',
 'home_team_score',
 'period',
 'postseason',
 'season',
 'status',
 'time',
 'visitor_team_score',
 'home_team_id',
 'home_team_abbreviation',
 'home_team_city',
 'home_team_conference',
 'home_team_division',
 'home_team_full_name',
 'home_team_name',
 'visitor_team_id',
 'visitor_team_abbreviation',
 'visitor_team_city',
 'visitor_team_conference',
 'visitor_team_division',
 'visitor_team_full_name',
 'visitor_team_name']

### STATS DATA :

In [13]:
#stats_df = fetch_data_with_counter(data_type="stats")[0]
stats_df = pd.read_csv("api_data/stats_data.csv", sep=";")

In [18]:
stats_df.dtypes

id                           int64
ast                        float64
blk                        float64
dreb                       float64
fg3_pct                    float64
fg3a                       float64
fg3m                       float64
fg_pct                     float64
fga                        float64
fgm                        float64
ft_pct                     float64
fta                        float64
ftm                        float64
min                         object
oreb                       float64
pf                         float64
pts                        float64
reb                        float64
stl                        float64
turnover                   float64
game.id                      int64
game.date                   object
game.home_team_id            int64
game.home_team_score         int64
game.period                  int64
game.postseason               bool
game.season                  int64
game.status                 object
game.time           

## Connecting to Postgres DataBase : 

### Inserting Data Into PLAYERS Table :

In [88]:
connection, cursor = None, None

tuples = [tuple(x) for x in players_df.to_numpy()]

cols = ",".join(players_df.columns.to_list()).replace(".", "_")

table_name = "players"

try : 
    connection = psycopg2.connect(user = "postgres", 
                                  password="postgrepass", 
                                  host="localhost", 
                                  port="5432", 
                                  database="nba")
    cursor = connection.cursor()

    create_table_script = f''' CREATE TABLE IF NOT EXISTS {table_name}(
                            id integer,
                            first_name varchar,
                            last_name varchar,
                            position varchar,
                            height_feet float,
                            height_inches float,
                            weight_pounds float,
                            team_id integer,
                            team_abbreviation varchar,
                            team_city varchar,
                            team_conference varchar,
                            team_division varchar,
                            team_name varchar,
                            team_full_name varchar
                            )'''
    
    cursor.execute(create_table_script)
    
    insert_data_script = f''' INSERT INTO players ({cols}) VALUES %s ON CONFLICT DO NOTHING ''' 
     
    # Execute the query with the data using execute_values() method
    extras.execute_values(cursor, insert_data_script, tuples)
    
    connection.commit()
    
except Exception as error:
    print(error)
finally :
    if cursor is not None: 
        cursor.close()
    if connection is not None: 
        connection.close()



### Inserting Data Into Teams Table :

In [19]:
connection, cursor = None, None

tuples = [tuple(x) for x in teams_df.to_numpy()]

cols = ",".join(teams_df.columns.to_list()).replace(".", "_")

table_name = "teams"

try : 
    connection = psycopg2.connect(user = "postgres", 
                                  password="postgrepass", 
                                  host="localhost", 
                                  port="5432", 
                                  database="nba")
    cursor = connection.cursor()

    create_table_script = f''' CREATE TABLE IF NOT EXISTS {table_name}(
                            id int,
                            abbreviation varchar,
                            city varchar,
                            conference varchar,
                            division varchar,
                            full_name varchar,
                            name varchar
                            )'''
    
    cursor.execute(create_table_script)
    
    insert_data_script = f"INSERT INTO {table_name} ({cols}) VALUES %s ON CONFLICT DO NOTHING"
     
    # Execute the query with the data using execute_values() method
    extras.execute_values(cursor, insert_data_script, tuples)
    
    connection.commit()
    
except Exception as error:
    print(error)
finally :
    if cursor is not None: 
        cursor.close()
    if connection is not None: 
        connection.close()



### Inserting Data Into Games Table :

In [30]:
connection, cursor = None, None

tuples = [tuple(x) for x in games_df.to_numpy()]

cols = ",".join(games_df.columns.to_list()).replace(".", "_")

table_name = "games"

try : 
    connection = psycopg2.connect(user = "postgres", 
                                  password="postgrepass", 
                                  host="localhost", 
                                  port="5432", 
                                  database="nba")
    cursor = connection.cursor()

    create_table_script = f''' CREATE TABLE IF NOT EXISTS {table_name}(
                            id int,
                            date varchar,
                            home_team_score int,
                            period int,
                            postseason boolean,
                            season int,
                            status varchar,
                            time varchar,
                            visitor_team_score int,
                            home_team_id int,
                            home_team_abbreviation varchar,
                            home_team_city varchar,
                            home_team_conference varchar,
                            home_team_division varchar,
                            home_team_full_name varchar,
                            home_team_name varchar,
                            visitor_team_id int,
                            visitor_team_abbreviation varchar,
                            visitor_team_city varchar,
                            visitor_team_conference varchar,
                            visitor_team_division varchar,
                            visitor_team_full_name varchar,
                            visitor_team_name varchar
                            )'''
    
    cursor.execute(create_table_script)
    
    insert_data_script = f"INSERT INTO {table_name} ({cols}) VALUES %s ON CONFLICT DO NOTHING"
     
    # Execute the query with the data using execute_values() method
    extras.execute_values(cursor, insert_data_script, tuples)
    
    connection.commit()
    
except Exception as error:
    print(error)
finally :
    if cursor is not None: 
        cursor.close()
    if connection is not None: 
        connection.close()



### Inserting Data Into Stats Table :

In [31]:
stats_df.dtypes

id                           int64
ast                        float64
blk                        float64
dreb                       float64
fg3_pct                    float64
fg3a                       float64
fg3m                       float64
fg_pct                     float64
fga                        float64
fgm                        float64
ft_pct                     float64
fta                        float64
ftm                        float64
min                         object
oreb                       float64
pf                         float64
pts                        float64
reb                        float64
stl                        float64
turnover                   float64
game.id                      int64
game.date                   object
game.home_team_id            int64
game.home_team_score         int64
game.period                  int64
game.postseason               bool
game.season                  int64
game.status                 object
game.time           

In [ ]:
connection, cursor = None, None

tuples = [tuple(x) for x in stats_df.to_numpy()]

cols = ",".join(stats_df.columns.to_list()).replace(".", "_")

table_name = "stats"

try : 
    connection = psycopg2.connect(user = "postgres", 
                                  password="postgrepass", 
                                  host="localhost", 
                                  port="5432", 
                                  database="nba")
    cursor = connection.cursor()

    create_table_script = f''' CREATE TABLE IF NOT EXISTS {table_name}(
                            id int,
                            date varchar,
                            home_team_score int,
                            period int,
                            postseason boolean,
                            season int,
                            status varchar,
                            time varchar,
                            visitor_team_score int,
                            home_team_id int,
                            home_team_abbreviation varchar,
                            home_team_city varchar,
                            home_team_conference varchar,
                            home_team_division varchar,
                            home_team_full_name varchar,
                            home_team_name varchar,
                            visitor_team_id int,
                            visitor_team_abbreviation varchar,
                            visitor_team_city varchar,
                            visitor_team_conference varchar,
                            visitor_team_division varchar,
                            visitor_team_full_name varchar,
                            visitor_team_name varchar
                            )'''
    
    cursor.execute(create_table_script)
    
    insert_data_script = f"INSERT INTO {table_name} ({cols}) VALUES %s ON CONFLICT DO NOTHING"
     
    # Execute the query with the data using execute_values() method
    extras.execute_values(cursor, insert_data_script, tuples)
    
    connection.commit()
    
except Exception as error:
    print(error)
finally :
    if cursor is not None: 
        cursor.close()
    if connection is not None: 
        connection.close()

